## IMPUTE INSANITY!:
### before we move onto neural networks we need to impute the NANs
#### How do we even know we've correctly imputed?
#### **Try out every impute methods and per feature set train the GBM and see which one got most importance**

### TODO
- #### [KNN , DeepLearner, other novel ways to impute](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)
  - KNN thoughts: we can perhaps use subset of features with no nans, but then the records will become similar, and decision trees already predict the same result for similar looking rows? then what's the point of imputing value same as similar looking rows?
- ### DON't Impute rather provide the Flag wether the feature has value or not, kinda like mask; we did this in Optiver (not the transformer sequence mask , but we have mask about weather the second/grouped seconds is missing or not)
  - #### [Results on passing information about missing values to neural networks](https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/)
- ### AMEND ABOVE: Impute as well as pass the missing Flag too, so neural network learns better
- ### Categorical Imputation: train on data with missing categorical values as target variable! **We can even use TEST set to do this!!!! yay!**
  - #### [Training to find missing categorical values](https://www.analyticsvidhya.com/blog/2021/04/how-to-handle-missing-values-of-categorical-variables/)

### IMPORTANT FLOAT16 aggregations return NONE!!! mean, sum most of it doesn't work :-/

## TODO:
- ### MULTILEVEL IMPUTATION DEFAULT
- ### LEVEL 1: out of all the variant+original; select the one with best amex scores DEFAULT_EMP1
    - #### This is partial nans
- ### Level 2: now that we have partial none df with best variants(including original with nans) ; TRAIN ML MODEL to have PREDICTED/BS variant
    - #### In the order of lowest missing values to highest missing values
    - #### we Target the feature with lowest amount on NAN as we have maximum training rows; then slowly cascade to more missing row features
    - #### once we have computed PREDICTED/BS variants for all of the features with nans
- ### Level 3: from DEFAULT_EMP1 vs PREDICTED variant select the one with maximum amex score (this is kinda like back to stage 1 but with improved train_df_wt)
    - #### repetetively select and choose the best varint out of the two
    - #### DEFAULT_EMP2 is generated!
    - #### Now there is a scope of repeating Level 2 again with using base_df of EMP2 and then going to EMP3,

In [1]:
WHERE_THIS = "local" # local|kaggle

BS_DATASET = "amex-default-prediction-binarysentient"
if WHERE_THIS == "kaggle":
    INPUT_PATH = "/kaggle/input/amex-default-prediction/"
    OUTPUT_PATH = "/kaggle/working/"
    TEMP_PATH = "/kaggle/temp/"
elif WHERE_THIS == "local":
    INPUT_PATH = "input/amex-default-prediction/"
    OUTPUT_PATH = "working/"
    TEMP_PATH = "temp/"
import os
BS_PATH = os.path.join(TEMP_PATH,BS_DATASET)
MODEL_PATH = "models/"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import os
from datetime import datetime
import torch
from multiprocessing import Pool

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
import plotly.express as pltex
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import impute_insanity
from impute_insanity import load_prepare_amex_dataset
import amex_bs_torch
from amex_bs_torch import AmexDefaultBSDataset
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk(INPUT_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input/amex-default-prediction/sample_submission.csv
input/amex-default-prediction/test_data.csv
input/amex-default-prediction/train_data.csv
input/amex-default-prediction/train_labels.csv
input/amex-default-prediction/.ipynb_checkpoints\sample_submission-checkpoint.csv


In [3]:
object_features = ['customer_ID']
discard_features = ['D_66', 'D_87']
datetime_features = ['S_2']
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']
target_features = ['target']
numeric_features = ['D_87', 'R_2', 'S_25', 'D_118', 'B_5', 'D_60', 'B_21', 'D_115', 'S_15', 'D_84', 'D_122', 'B_9', 'B_33', 'D_53', 'R_24', 'D_94', 'D_56', 'D_139', 'R_28', 'B_3', 'S_20', 'B_31', 'D_133', 'B_2', 'S_12', 'D_71', 'D_96', 'S_7', 'D_72', 'B_36', 'B_41', 'S_5', 'D_41', 'R_22', 'R_8', 'D_140', 'D_47', 'D_89', 'P_2', 'R_19', 'D_59', 'B_23', 'S_3', 'D_145', 'D_103', 'B_19', 'R_20', 'D_73', 'D_136', 'D_141', 'D_142', 'B_22', 'D_46', 'B_29', 'B_25', 'D_128', 'B_18', 'D_86', 'D_109', 'B_8', 'B_17', 'R_17', 'B_12', 'D_54', 'D_74', 'S_16', 'B_6', 'D_49', 'D_80', 'S_8', 'B_7', 'D_144', 'B_27', 'B_26', 'R_25', 'R_23', 'D_82', 'D_111', 'B_10', 'D_113', 'R_4', 'D_48', 'R_26', 'S_23', 'B_11', 'D_104', 'D_134', 'D_79', 'P_3', 'D_132', 'D_137', 'D_135', 'B_28', 'D_88', 'S_13', 'D_51', 'D_61', 'D_75', 'D_69', 'R_16', 'S_6', 'S_17', 'D_93', 'B_20', 'D_112', 'D_123', 'D_130', 'B_1', 'D_78', 'D_92', 'S_27', 'D_44', 'B_16', 'R_5', 'D_43', 'S_18', 'B_15', 'D_39', 'D_50', 'D_55', 'S_9', 'D_105', 'D_70', 'R_18', 'D_125', 'D_58', 'S_24', 'D_110', 'D_42', 'R_6', 'D_81', 'R_7', 'D_138', 'D_52', 'R_27', 'D_124', 'D_45', 'D_91', 'D_108', 'S_22', 'B_14', 'D_83', 'R_13', 'S_19', 'D_131', 'R_21', 'B_40', 'R_3', 'D_65', 'B_13', 'D_129', 'D_119', 'B_32', 'R_9', 'B_24', 'D_127', 'D_106', 'D_102', 'R_1', 'R_14', 'B_37', 'D_107', 'R_15', 'R_12', 'P_4', 'R_10','D_121', 'R_11', 'S_11', 'D_76', 'D_143', 'B_39', 'B_42', 'D_62', 'B_4', 'S_26', 'D_77']
    

In [4]:
## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def fast_amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

In [5]:
use_cuda = torch.cuda.is_available()
# use_cuda = False
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [6]:
device

device(type='cuda', index=0)

In [6]:
amexbsdataset = AmexDefaultBSDataset(total_chunks=128, lazy_load=False, device=device)

LOAD EAZZZZZ
NON LAZY LOAD
['train_data_emp1_nn_chunk_121', 'train_data_emp1_nn_chunk_38', 'train_data_emp1_nn_chunk_3', 'train_data_emp1_nn_chunk_63', 'train_data_emp1_nn_chunk_85', 'train_data_emp1_nn_chunk_109', 'train_data_emp1_nn_chunk_5', 'train_data_emp1_nn_chunk_14', 'train_data_emp1_nn_chunk_46', 'train_data_emp1_nn_chunk_116', 'train_data_emp1_nn_chunk_125', 'train_data_emp1_nn_chunk_12', 'train_data_emp1_nn_chunk_127', 'train_data_emp1_nn_chunk_103', 'train_data_emp1_nn_chunk_114', 'train_data_emp1_nn_chunk_18', 'train_data_emp1_nn_chunk_115', 'train_data_emp1_nn_chunk_59', 'train_data_emp1_nn_chunk_19', 'train_data_emp1_nn_chunk_72', 'train_data_emp1_nn_chunk_8', 'train_data_emp1_nn_chunk_106', 'train_data_emp1_nn_chunk_1', 'train_data_emp1_nn_chunk_27', 'train_data_emp1_nn_chunk_20', 'train_data_emp1_nn_chunk_95', 'train_data_emp1_nn_chunk_30', 'train_data_emp1_nn_chunk_34', 'train_data_emp1_nn_chunk_98', 'train_data_emp1_nn_chunk_120', 'train_data_emp1_nn_chunk_32', 'trai

In [8]:
torch.cuda.empty_cache()
gc.collect()

0

In [7]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 13):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [7]:
class AmexBSModel(nn.Module):
    def __init__(self, category_embedding_dimensions=3, mode="train"):
        """
        Args:
            mode: train|feature_extractor
        """
        super(AmexBSModel,self).__init__()
        self.mode=mode
        self.sequence_length = 13
        # need categorical emeddings for each cat_features
        # features + all the embeddings go into input of transformer
        # transformer output goes to the linear layers
        # output of linear layer should be binary classification/probability
        # each embeddig has the number of classes it can have, then we add 1 as we'll have extra category for masked data
        self.cat_embedding_dimensions = category_embedding_dimensions
        self.cat_encoders = amex_bs_torch.get_generate_cat_encoders()
        self.cat_embedding_layer = nn.ModuleList([nn.Embedding(len(self.cat_encoders[x].classes_)+1, self.cat_embedding_dimensions) for x  in cat_features])
        
        
        self.transformer_input_dimension = len(self.cat_embedding_layer)*self.cat_embedding_dimensions + len(self.cat_embedding_layer) + len(numeric_features)*2 + 2 # we'll have custom feature for months in system as feature
        # self.positional_ecoding = PositionalEncoding(self.transformer_input_dimension, max_len=self.sequence_length)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.transformer_input_dimension, batch_first=True, nhead=1, dim_feedforward=1024, dropout=0.25, activation='gelu')
        self.encoder_stack = nn.TransformerEncoder(encoder_layer, num_layers=1)
        self.gru_hidden_size = self.transformer_input_dimension
        self.gru_stack = nn.GRU(input_size=self.transformer_input_dimension, hidden_size=self.gru_hidden_size, num_layers=1, batch_first=True)
        self.linear_stack = nn.Sequential(
            nn.Linear(self.transformer_input_dimension*self.sequence_length, 256),
            # nn.Linear(self.gru_hidden_size, 256),
            # nn.Linear(self.transformer_input_dimension*3, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 64),
             nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(64, self.gru_hidden_size)
        )
        self.post_transformer = nn.Sequential(
            nn.Linear(self.transformer_input_dimension*self.sequence_length, self.gru_hidden_size)
            # nn.Linear(self.gru_hidden_size, 256),
            # nn.Linear(self.transformer_input_dimension*3, 256),
        )
        self.mega_fusion = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.gru_hidden_size*3, 512),
            # nn.Linear(self.gru_hidden_size, 256),
            # nn.Linear(self.transformer_input_dimension*3, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(512, 64),
             nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        self.linear_after_transformer = nn.Sequential(
            
            nn.Linear(self.transformer_input_dimension, 512),
            # nn.Linear(self.gru_hidden_size, 256),
            # nn.Linear(self.transformer_input_dimension*3, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, 64),
             nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        # we have 3*
        pass
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        sequence_features = x['sequence_features'].type(torch.float32).to(device)
        # shape (batch_size, sequence_length(13))
        datatheremask = (x['sequence_mask'].type(torch.FloatTensor).to(device)*-1)+1
        sequence_mask = x['sequence_mask'].type(torch.bool).to(device)
        
        
        cat_features_embedded = []
        # cat_features_nan_mask = []
        for idx, cat_feature in enumerate(cat_features):
            # print(sequence_features[cat_feature].shape)
            # shape (batch_size, seqlen(13), self.cat_embedding_dimension(2))
            cat_feature_embedded_ = self.cat_embedding_layer[idx](sequence_features[:,:,idx*2].type(torch.int32)) #
            cat_features_embedded.append(cat_feature_embedded_)
            cat_features_embedded.append(sequence_features[:,:,(idx*2)+1].unsqueeze(2))
        
        transformer_input_features = torch.cat(cat_features_embedded+[sequence_features[:,:,len(cat_features)*2:]], dim=2)
        # print(cat_features_embedded.shape, cat_features_embedded)
        
#         numeric_features_embedded = []
#         for idx, numeric_feature in enumerate(numeric_features):
#             numeric_features_embedded.append(sequence_features[numeric_feature].to(device)) #.type(torch.FloatTensor)
        
#         numeric_features_embedded = torch.stack(numeric_features_embedded, dim=2)
        
        # out custom reversed months in system encoding, last entry gets 1 value while first entry gets increasing value far from last value,
        maxmonthinsystem, maxmonthindex = torch.max(datatheremask.cumsum(1), dim=1)
        monthsinsystem_pos_encoding = (((maxmonthinsystem.unsqueeze(1) - datatheremask.cumsum(1))+1) / 1000.0) * datatheremask
        transformer_input_features_wpos = torch.add(transformer_input_features, monthsinsystem_pos_encoding.unsqueeze(2))
        
        # transformer_input_features = self.positional_ecoding(transformer_input_features.view(self.sequence_length, -1, self.transformer_input_dimension)).view(-1, self.sequence_length, self.transformer_input_dimension)
        
        # linear_stack_output = self.linear_stack(transformer_input_features.reshape(-1, self.transformer_input_dimension*self.sequence_length))
                                                
        transformer_output = self.encoder_stack(transformer_input_features, src_key_padding_mask=sequence_mask)
        # transformer_output = transformer_output.reshape(-1, self.transformer_input_dimension*self.sequence_length)
        transformer_output = transformer_output.mean(dim=1)
        # transformer_output = self.post_transformer(transformer_output)
        # transformer_output = torch.cat([transformer_output[:,0,:], transformer_output[:,6,:], transformer_output[:,-1,:]], dim=1)
        
        _, gru_output = self.gru_stack(transformer_input_features, transformer_output.view(1,-1,self.gru_hidden_size))
        gru_output = gru_output[-1,:,:].reshape(-1, self.gru_hidden_size)
        
        if self.mode == 'feature_extractor':
            return gru_output
        
        # print(linear_stack_output.shape, transformer_output.shape, gru_output.shape)
        # final_output = self.mega_fusion(torch.cat([linear_stack_output, transformer_output, gru_output], dim=1))
        final_output = self.linear_after_transformer(gru_output)
        
        # print(x['sequence_mask'].shape)
        return final_output
AmexBSModel()

AmexBSModel(
  (cat_embedding_layer): ModuleList(
    (0): Embedding(4, 3)
    (1): Embedding(9, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(8, 3)
    (5): Embedding(4, 3)
    (6): Embedding(4, 3)
    (7): Embedding(7, 3)
    (8): Embedding(5, 3)
    (9): Embedding(8, 3)
  )
  (encoder_stack): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=396, out_features=396, bias=True)
        )
        (linear1): Linear(in_features=396, out_features=1024, bias=True)
        (dropout): Dropout(p=0.25, inplace=False)
        (linear2): Linear(in_features=1024, out_features=396, bias=True)
        (norm1): LayerNorm((396,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((396,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.25, inplace=False)
        (dropout2): Dropout(p=0.25, inplace=False)
    

In [9]:
def get_weighted_random_sampler(indices=None):
    train_labels = load_prepare_amex_dataset("train_labels")
    
    # we have class 0 and 1, 1 class is 25% ish, so to make them equal we turn 0,1 to 0,0.5 then 0.25,0.75 so that class 1 is 75% scale while class 0 is 25% scale

    # we set replacement to False so that we can pass through every samples in one epoch; we also now have to reduce length by 4 otherwise when all 1s are used then only 0s will be left as target
    thetarget = None
    if indices is None:
        thetarget = train_labels['target']
    else:
        thetarget = train_labels['target'].loc[indices]
    sampler = WeightedRandomSampler(((thetarget*0.5)+0.25).values, len(thetarget)//4, replacement=False)
    del train_labels
    return sampler

In [10]:
dataset_size = len(amexbsdataset)
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(amexbsdataset, [train_size, test_size])

In [22]:
del amex_bs_model
del loss_function
del loss
del y_true
del y_pred
del optimizer
del train_dataloader
del test_dataloader
torch.cuda.empty_cache()
gc.collect()


1277

In [ ]:
max_epochs = 200
batch_size = 512
accumulation_steps = 1
amex_bs_model = AmexBSModel().to(device)
optimizer = torch.optim.AdamW(amex_bs_model.parameters(), lr=0.00002)
loss_function = torch.nn.BCEWithLogitsLoss().to(device)
# loss_function = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(3.0)).to(device)

# loss_function = torch.nn.BCELoss()

# loss_function = torch.nn.CrossEntropyLoss(torch.tensor([1.0,4.0], dtype=torch.float32)).to(device)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=4096, pin_memory=True)#, sampler=get_weighted_random_sampler(test_dataset.indices))


for epoch in range(max_epochs):
    print("--- EPOCH :", epoch+1)
     #, sampler=get_weighted_random_sampler(train_dataset.indices))
    
    # if epoch==0:
    #     batch_size=4096
    #     accumulation_steps=4
    # elif epoch==1:
    #     batch_size=4096
    #     accumulation_steps=1
    # elif epoch>=2:
    #     batch_size=1024
    #     accumulation_steps=1
    # elif epoch>=5:
    #     batch_size=128
    #     accumulation_steps=1
    # elif epoch>=10:
    #     batch_size=64
    #     accumulation_steps=1
    
   
    current_samples = 0
    current_losses = []
    
    for train_batch_idx, (Feature_X, y_true) in enumerate(train_dataloader): 
        # print(train_batch_idx)
        y_pred = amex_bs_model(Feature_X)
        y_true = y_true.type(torch.FloatTensor).to(device)
        loss = loss_function(y_pred, y_true.reshape(-1,1))
        loss.backward()  
        if (train_batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        current_samples += batch_size
        current_losses.append(loss.item())
        if current_samples < (len(train_dataset)/5) :
            # we display 10% marks
            continue
        current_samples = 0
  
        print(f"Loss:  {np.mean(current_losses):0.6f}")
        current_losses = []
    
    amex_bs_model.eval()
    amex_scores = []
    for train_batch_idx, (Feature_X, y_true) in enumerate(test_dataloader): 

        y_pred = amex_bs_model(Feature_X).reshape(-1).cpu().detach().numpy()
        y_true = y_true.detach().numpy()

        amex_scores.append(fast_amex_metric(y_true, y_pred))
    print("-------------------------")
    print(f"AMEX SCORE: {np.mean(amex_scores):0.5f}")
    print("-------------------------")
    amex_bs_model.train()
    
    
    os.makedirs(MODEL_PATH, exist_ok=True)
    torch.save(amex_bs_model.state_dict(), os.path.join(MODEL_PATH,f"{np.mean(amex_scores):0.5f}_amex_default_bs_model_epoch_{epoch}.pt"))
    
    # print("Batch, train_batch_idx", train_batch_idx)
    # print(feature_y.sum(), feature_y)
    # input()
    # if train_batch_idx>40:
    #     break

    

In [190]:
y_pred

array([-1.9248886 ,  1.4796506 , -0.36563623, ..., -0.32325116,
       -2.1727817 ,  1.8644756 ], dtype=float32)

<All keys matched successfully>

In [8]:
amexbsdataset_test = AmexDefaultBSDataset(mode="test",total_chunks=128, lazy_load=False, device=device)

LOAD EAZZZZZ
NON LAZY LOAD
['test_data_emp1_nn_chunk_28', 'test_data_emp1_nn_chunk_41', 'test_data_emp1_nn_chunk_80', 'test_data_emp1_nn_chunk_123', 'test_data_emp1_nn_chunk_36', 'test_data_emp1_nn_chunk_16', 'test_data_emp1_nn_chunk_87', 'test_data_emp1_nn_chunk_13', 'test_data_emp1_nn_chunk_102', 'test_data_emp1_nn_chunk_2', 'test_data_emp1_nn_chunk_23', 'test_data_emp1_nn_chunk_53', 'test_data_emp1_nn_chunk_54', 'test_data_emp1_nn_chunk_108', 'test_data_emp1_nn_chunk_84', 'test_data_emp1_nn_chunk_97', 'test_data_emp1_nn_chunk_7', 'test_data_emp1_nn_chunk_104', 'test_data_emp1_nn_chunk_50', 'test_data_emp1_nn_chunk_66', 'test_data_emp1_nn_chunk_57', 'test_data_emp1_nn_chunk_105', 'test_data_emp1_nn_chunk_95', 'test_data_emp1_nn_chunk_12', 'test_data_emp1_nn_chunk_101', 'test_data_emp1_nn_chunk_120', 'test_data_emp1_nn_chunk_121', 'test_data_emp1_nn_chunk_59', 'test_data_emp1_nn_chunk_19', 'test_data_emp1_nn_chunk_64', 'test_data_emp1_nn_chunk_67', 'test_data_emp1_nn_chunk_94', 'test_

In [9]:
amex_bs_model = AmexBSModel()
amex_bs_model.load_state_dict(torch.load(os.path.join(MODEL_PATH,"0.79105_amex_default_bs_model_epoch_9.pt")))
amex_bs_model.eval()

AmexBSModel(
  (cat_embedding_layer): ModuleList(
    (0): Embedding(4, 3)
    (1): Embedding(9, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(8, 3)
    (5): Embedding(4, 3)
    (6): Embedding(4, 3)
    (7): Embedding(7, 3)
    (8): Embedding(5, 3)
    (9): Embedding(8, 3)
  )
  (encoder_stack): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=396, out_features=396, bias=True)
        )
        (linear1): Linear(in_features=396, out_features=1024, bias=True)
        (dropout): Dropout(p=0.25, inplace=False)
        (linear2): Linear(in_features=1024, out_features=396, bias=True)
        (norm1): LayerNorm((396,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((396,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.25, inplace=False)
        (dropout2): Dropout(p=0.25, inplace=False)
    

In [21]:
#, dtype={f"bs_{x}":'float32' for x in range(amex_bs_model.gru_hidden_size)})

,customer_ID,bs_0,bs_1,bs_2,bs_3,bs_4,bs_5,bs_6,bs_7,bs_8,...,bs_386,bs_387,bs_388,bs_389,bs_390,bs_391,bs_392,bs_393,bs_394,bs_395


In [11]:
amex_bs_model.mode = "feature_extractor"
amex_bs_model.to(device)
amex_bs_model.eval()
dataloader_test = DataLoader(amexbsdataset_test, shuffle=False, batch_size=4096, pin_memory=True)
customer_ids = []
ypreds = []
features_df = pd.DataFrame(columns=['customer_ID']+[f"bs_{x}" for x in range(amex_bs_model.gru_hidden_size)])
for train_batch_idx, (Feature_X, y_true) in enumerate(dataloader_test):
    Feature_X['sequence_features'] = torch.nan_to_num(Feature_X['sequence_features'])
    y_pred = amex_bs_model(Feature_X)
    # print(y_pred)
    feature_columns = {}
    feature_columns['customer_ID'] = Feature_X['customer_ID']
    for idx, feature in enumerate(y_pred.permute(1,0).unbind(dim=0)):
        feature_columns[f"bs_{idx}"] = feature.cpu().detach().numpy()
    
    features_df = pd.concat([features_df, pd.DataFrame(feature_columns)], ignore_index=True)
    # input(features_df)
    if torch.isnan(y_pred).any():
        print("______ NANANANANA ____________")
    y_pred = y_pred.reshape(-1).detach().tolist()
    
    ypreds += y_pred
    # print(y_pred)
    customer_id = Feature_X['customer_ID']
    customer_ids += customer_id
    # print(customer_ids)
    # input()

In [12]:
features_df.to_parquet(os.path.join(BS_PATH,"test_data_bs_features_extracted.parquet"))

In [17]:
pd.DataFrame({'customer_ID':customer_ids,'prediction':ypreds}).to_csv("submission.csv",index=False)

In [6]:
load_prepare_amex_dataset("test_data_emp1_nn_chunk_126")

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136_nan_mask,D_137_nan_mask,D_138_nan_mask,D_139_nan_mask,D_140_nan_mask,D_141_nan_mask,D_142_nan_mask,D_143_nan_mask,D_144_nan_mask,D_145_nan_mask
0,fbfa8ef66af90ba332c1c024fea9e2ad89a34d3692a07b...,2018-04-23,0.990080,0.002432,0.036668,1.009193,0.004333,0.109592,0.009624,0.008216,...,True,True,True,False,False,False,False,False,False,False
1,fbfa8ef66af90ba332c1c024fea9e2ad89a34d3692a07b...,2018-05-16,0.933168,0.000008,0.036715,1.007722,0.007168,0.097053,0.001303,0.006442,...,True,True,True,False,False,False,False,False,False,False
2,fbfa8ef66af90ba332c1c024fea9e2ad89a34d3692a07b...,2018-06-01,0.934100,0.123914,0.033306,1.001847,0.007540,0.098578,0.006818,0.008657,...,True,True,True,False,False,False,False,False,False,False
3,fbfa8ef66af90ba332c1c024fea9e2ad89a34d3692a07b...,2018-07-02,0.896938,0.152745,0.112452,1.009726,0.006455,0.081935,0.005706,0.006653,...,True,True,True,False,False,False,False,False,False,False
4,fbfa8ef66af90ba332c1c024fea9e2ad89a34d3692a07b...,2018-08-20,0.990730,0.000764,0.012469,1.001405,0.003212,0.086514,0.007664,0.002409,...,True,True,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88951,fdfb1a337f25534b21dfe27e582878243d7ef5db838019...,2019-06-19,0.530724,0.148674,0.092000,0.087423,0.007374,0.303600,0.009540,0.110265,...,False,False,False,False,False,False,False,False,False,False
88952,fdfb1a337f25534b21dfe27e582878243d7ef5db838019...,2019-07-15,0.517142,0.009457,0.084325,0.085383,0.001179,0.274656,0.002791,0.105405,...,False,False,False,False,False,False,False,False,False,False
88953,fdfb1a337f25534b21dfe27e582878243d7ef5db838019...,2019-08-18,0.532790,0.066512,0.086585,0.091485,0.007870,0.451863,0.000864,0.160311,...,False,False,False,False,False,False,False,False,False,False
88954,fdfb1a337f25534b21dfe27e582878243d7ef5db838019...,2019-09-14,0.531693,0.008981,0.078207,0.092427,0.005957,0.467106,0.006395,0.153611,...,False,False,False,False,False,False,False,False,False,False


In [32]:
nanmask = load_prepare_amex_dataset("train_data_nan_mask")

In [44]:
nanmask.rename(lambda x: x+"_nan" if x!="customer_ID" else x, axis=1)
nanmask

,customer_ID,S_2_nan,P_2_nan,D_39_nan,B_1_nan,B_2_nan,R_1_nan,S_3_nan,D_41_nan,B_3_nan,...,D_136_nan,D_137_nan,D_138_nan,D_139_nan,D_140_nan,D_141_nan,D_142_nan,D_143_nan,D_144_nan,D_145_nan
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
5531447,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
5531448,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False
5531449,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,False,False,False


```
Data: EMP1_nn
Features: Sequence_BSemp1, SequenceNANMASKS
Model: BS Transformer -> TOUT As HiddenInitialState -> GRU, Linear , 256batch size
Model: 0.79105_amex_default_bs_model_epoch_9
```